## Generate synthetic data for DPO

This notebook creates a synthetic dataset of classified ads and scores them in pairs.

In [1]:
#%pip install --upgrade --quiet google-genai

In [2]:
MODEL_ID = "models/gemini-2.0-flash"

import os
from dotenv import load_dotenv
load_dotenv("../keys.env");
assert os.environ["GEMINI_API_KEY"][:2] == "AI",\
       "Please specify the GEMINI_API_KEY access token in keys.env file"

In [3]:
from google import genai
from google.genai import types

client = genai.Client(api_key=os.getenv('GEMINI_API_KEY'))

## Create an ad for an item

In [4]:
import random
def create_classified_ad(item: str, price: str) -> str:
    prompt = f"""
    You are writing a classified ad for used items in a neighborhood online forum. Write a one paragraph description for
    a {item} priced at {price}
    """
    response = client.models.generate_content(
        model=MODEL_ID,
        contents=[prompt],
        config=types.GenerateContentConfig(
            max_output_tokens=500,
            temperature=random.uniform(0.2, 0.9),
        )
    )
    return response.text

create_classified_ad("3-year old Specialized bike", "$300")

'Lightly used, 3-year-old Specialized bike in excellent condition, perfect for commuting or recreational rides. This reliable and comfortable bike has been well-maintained and features smooth-shifting gears and responsive brakes. Minor cosmetic wear and tear, but mechanically sound and ready to ride. Asking $300. Serious inquiries only, please. Local pickup preferred.\n'

In [5]:
example_ad = create_classified_ad("3-year old Specialized bike", "$300")
example_ad

'Lightly used, 3-year-old Specialized hybrid bike in excellent condition, perfect for commuting or recreational rides. Features a comfortable saddle, smooth-shifting gears, and responsive brakes. Well-maintained and ready to ride. Asking $300. Serious inquiries only, please. Local pickup in [Neighborhood Name] preferred.\n'

## Score an ad

In [6]:
from dataclasses import dataclass

@dataclass
class AdScore:
    clarity: int
    audience_targeting: int
    readability: int
    contact_info: int
    
    def total_score(self) -> int:
        return self.clarity + self.audience_targeting + self.readability + self.contact_info

def score_ad(ad: str) -> AdScore:
    prompt = f"""
    You are a professor of advertising at a business school. Score the following classified ad based on the following criteria:
    * Is it clear what's being sold? Age, brand, price, and condition are important. (10 points)
    * Does it target the most relevant audience for the item? Is it persuasive to that audience? (10 points)
    * Is it concise and easy to read? (10 points)
    * Does it include contact information? Ideally, the ad specifies the preferred means of communication. (10 points)
    
    Ad:
    {ad}
    """
    response = client.models.generate_content(
        model=MODEL_ID,
        contents=[prompt],
        config=types.GenerateContentConfig(
            temperature=0.1,
            response_mime_type='application/json',
            response_schema=AdScore
        )
    )
    
    return response.parsed

In [7]:
example_score = score_ad(example_ad)
example_score

AdScore(clarity=9, audience_targeting=8, readability=9, contact_info=7)

In [8]:
example_score.total_score()

33

## Generate data

In [9]:
def create_preference_example(item: str, price: str) -> dict:
    ad1 = create_classified_ad(item, price)
    ad2 = create_classified_ad(item, price)
    score1 = score_ad(ad1).total_score()
    score2 = score_ad(ad2).total_score()
    
    preference_example = {
        "prompt": f"""You are writing a classified ad for used items in a neighborhood online forum. Write a one paragraph description for a {item} priced at {price}"""
    }
    
    if score1 > score2:
        preference_example['chosen'] = ad1
        preference_example['rejected'] = ad2
    else:
        preference_example['chosen'] = ad2
        preference_example['rejected'] = ad1
    
    return preference_example

create_preference_example("3-year old Specialized bike", "$300")

{'prompt': 'You are writing a classified ad for used items in a neighborhood online forum. Write a one paragraph description for a 3-year old Specialized bike priced at $300',
 'chosen': 'Lightly used, 3-year-old Specialized mountain bike in excellent condition. Perfect for hitting the trails or cruising around town. Features a sturdy aluminum frame, reliable Shimano components, and responsive brakes. Regularly maintained and ready to ride. Asking $300. Serious inquiries only, please contact [Your Contact Info] to schedule a viewing.\n',
 'rejected': "Get ready to ride! Selling my well-maintained, 3-year-old Specialized bike for $300. Perfect for commuting, weekend adventures, or just cruising around the neighborhood. It's in great condition with some minor cosmetic wear, recently tuned up, and ready to roll. Asking $300 OBO. Serious inquiries only, please. Contact me to arrange a viewing!\n"}

In [10]:
import json

# generated using Gemini with the prompt
"""
Create a synthetic dataset of 10 items for sale in a garage sale. 
Items should be in the format (item_description, cost). 
For example ("Like-new copy of Chinua Achebe's Things Fall Apart", "$5"). 
# [optional] Items should be somewhat expensive, and include details such as brand name where possible.
# [optional] Items should be in the 10-20 dollar range and be unique. Not a collection of things.
"""
items_for_sale = [
    ("3-year old Specialized road bike", "$300"),
    ("Amazing Spider-Man 361", "$200"),
    ("Like-new copy of Chinua Achebe's Things Fall Apart", "$5"),
    ("6-piece Le Creuset cookware set in good condition", "$800"),
    ("Well-used kids tricycle", "$20"),
    ("1990 vintage pair of Levi's 501 jeans size is Men's 32x32 in good condition", "$50"),
    ("Vintage Pyrex mixing bowl set (3 bowls)", "$15"),
    ("Hardwood rocking chair (good condition)", "$50"),
    ("Kids' bicycle (16-inch wheels)", "$25"),
    ("Set of 4 dining chairs (wooden)", "$40"),
    ("Large area rug (floral pattern)", "$30"),
    ("Electric coffee maker (like new)", "$10"),
    ("Box of assorted DVDs (mostly action movies)", "$20"),
    ("Ceramic table lamp (with shade)", "$12"),
    ("Gardening tools set (shovel, rake, trowel)", "$25"),
    ("Board game collection (Monopoly, Scrabble, Clue)", "$35"),
    ("Antique dresser with mirror (restored)", "$250"),
    ("Solid oak dining table with 6 chairs", "$400"),
    ("Leather sofa (minor wear)", "$300"),
    ("Vintage record player (working condition)", "$175"),
    ("Collection of antique books (various genres)", "$200"),
    ("High-end road bike (carbon fiber frame)", "$800"),
    ("Hand-knotted Persian rug (small size)", "$450"),
    ("Original oil painting (landscape scene)", "$350"),
    ("Set of vintage china (complete set)", "$150"),
    ("Designer handbag (like new)", "$200"),
    ("Ethan Allen Armchair (excellent condition, dark green leather)", "$400"),
    ("Bose QuietComfort 35 Headphones (wireless, noise-canceling, like new)", "$150"),
    ("Samsung 55-inch Smart TV (4K UHD, excellent picture)", "$350"),
    ("KitchenAid Stand Mixer (Artisan series, red, with attachments)", "$200"),
    ("Breville Barista Express Espresso Machine (stainless steel)", "$300"),
    ("Apple iPad Pro 11-inch (64GB, Wi-Fi, Space Gray)", "$450"),
    ("Sony Alpha a6000 Mirrorless Camera (with two lenses)", "$500"),
    ("Ray-Ban Aviator Sunglasses (gold frame, polarized lenses)", "$100"),
    ("Michael Kors Leather Handbag (large tote, black)", "$150"),
    ("Baccarat Crystal Vase (small size, excellent condition)", "$250"),
    ("Hand-painted ceramic flower pot (with drainage hole)", "$12"),
    ("Vintage rotary phone (working condition, bright red)", "$18"),
    ("Framed print of a local landscape painting", "$15"),
    ("Set of 4 hand-blown glass coasters", "$10"),
    ("Woven bamboo storage basket (with lid)", "$14"),
    ("Small, decorative brass elephant figurine", "$16"),
    ("Hardcover cookbook: The Joy of Cooking", "$12"),
    ("Like-new copy of National Geographic magazine (special edition)", "$10"),
    ("Set of 2 vintage Pyrex coffee mugs (in original box)", "$20"),
    ("Hand-carved wooden serving spoon", "$15")
]

def write_jsonl(num_examples: int, filename: str):
    examples = []
    for iter in range(num_examples):
        print(iter, end=" ... ")
        item, price = random.choice(items_for_sale)
        example = create_preference_example(item, price)
        examples.append(example)
    
    with open(filename, "w") as ofp:
        for example in examples:
            json.dump(example, ofp)
            ofp.write('\n')
            
    return examples, filename

In [11]:
examples, filename = write_jsonl(2, "ad_preference_dataset.jsonl")

0 ... 1 ... 

In [12]:
len(examples)

2

In [13]:
!head *.jsonl

{"prompt": "You are writing a classified ad for used items in a neighborhood online forum. Write a one paragraph description for a Set of 2 vintage Pyrex coffee mugs (in original box) priced at $20", "chosen": "Calling all vintage lovers! I'm selling a set of two vintage Pyrex coffee mugs, still nestled in their original box. These classic mugs are in great condition and ready to add a touch of retro charm to your morning routine. Asking $20 for the set. Perfect for collectors or anyone who appreciates a bit of nostalgia with their coffee! PM me if interested.\n", "rejected": "Calling all vintage lovers! I'm selling a set of two pristine Pyrex coffee mugs, still nestled in their original box. These retro beauties are in excellent condition and ready to add a touch of mid-century charm to your morning coffee routine. Asking $20 for the set. Don't miss out on this rare find!\n"}
{"prompt": "You are writing a classified ad for used items in a neighborhood online forum. Write a one paragra

## Do it

In [14]:
write_jsonl(len(items_for_sale)*10, "ad_preference_dataset.jsonl");

0 ... 1 ... 2 ... 3 ... 4 ... 5 ... 6 ... 7 ... 8 ... 9 ... 10 ... 11 ... 12 ... 13 ... 14 ... 15 ... 16 ... 17 ... 18 ... 19 ... 20 ... 21 ... 22 ... 23 ... 24 ... 25 ... 26 ... 27 ... 28 ... 29 ... 30 ... 31 ... 32 ... 33 ... 34 ... 35 ... 36 ... 37 ... 38 ... 39 ... 40 ... 41 ... 42 ... 43 ... 44 ... 45 ... 46 ... 47 ... 48 ... 49 ... 50 ... 51 ... 52 ... 53 ... 54 ... 55 ... 56 ... 57 ... 58 ... 59 ... 60 ... 61 ... 62 ... 63 ... 64 ... 65 ... 66 ... 67 ... 68 ... 69 ... 70 ... 71 ... 72 ... 73 ... 74 ... 75 ... 76 ... 77 ... 78 ... 79 ... 80 ... 81 ... 82 ... 83 ... 84 ... 85 ... 86 ... 87 ... 88 ... 89 ... 90 ... 91 ... 92 ... 93 ... 94 ... 95 ... 96 ... 97 ... 98 ... 99 ... 100 ... 101 ... 102 ... 103 ... 104 ... 105 ... 106 ... 107 ... 108 ... 109 ... 110 ... 111 ... 112 ... 113 ... 114 ... 115 ... 116 ... 117 ... 118 ... 119 ... 120 ... 121 ... 122 ... 123 ... 124 ... 125 ... 126 ... 127 ... 128 ... 129 ... 130 ... 131 ... 132 ... 133 ... 134 ... 135 ... 136 ... 137 ... 138 ..

In [15]:
!wc -l *.jsonl

460 ad_preference_dataset.jsonl
